# Imports

In [1]:
%load_ext autoreload

%autoreload 2

# Implementation DeepLab v3+ 

In [2]:
import os
import cv2
import numpy as np
from glob import glob
from scipy.io import loadmat
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2023-09-26 09:43:12.049836: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 09:43:13.024145: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib/python3.10/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-09-26 09:43:13.024272: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LI

In [3]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

def load_and_preprocess_data(image_path, mask_path=None):
    """Function to load images from local storage and preprocess them
        Args:
            image_path: list of all image paths
            mask_path: list of all mask paths (None for tets/predict sets)
    """
    # Load and preprocess the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(images=image, size=[512, 512])
    image = image/255

    # Load and preprocess the mask image
    if mask_path is not None:
        mask = tf.io.read_file(mask_path)
        mask = tf.image.decode_png(mask, channels=1)
        mask = tf.image.resize(mask, [512, 512])
        mask = tf.math.greater(mask, 127)
        mask = tf.cast(mask, tf.float32)        
        return image, mask
    else:
        return image

def create_datasets(save_path, set="train", test_size=.1, batch_size=64, data_size=1, val_size=.3):
    """Function to create dataset batches
        Args:
            save_path: path of images
            set: kind of input dataset --> train, test or predict (save_path for predict is end folder)
            test_size: size of test_set, ignored for set in ["predict", "test"] --> float in range (0,1)
            batch_size= size of batches --> int
            data_size: size of original data to be used --> float in range (0,1), 1 = total data
            val_size: size of validation set, ignored for set in ["predict", "test"] --> float in range (0,1)
    """
    image_dir = f'{save_path}/{set}/images'
    mask_dir = f'{save_path}/{set}/gt' if set == "train" else None

    image_path = [os.path.join(image_dir, filename) for filename in os.listdir(image_dir)]
    mask_path = [os.path.join(mask_dir, filename) for filename in os.listdir(mask_dir)] if mask_dir else None
    
    #test split
    image_path_rest, image_path_test, mask_path_rest, mask_path_test = train_test_split(image_path, mask_path, test_size=test_size, random_state=42) if test_size != 0 else (None, None, None, None)

    #data size split
    image_path_subset, _, mask_path_subset, _ = train_test_split(image_path_rest, mask_path_rest, train_size=data_size) if data_size != 1 else (None, None, None, None)
    if data_size == 1:
        image_path_subset = image_path_rest
        mask_path_subset = mask_path_rest
    
    #train val split
    image_path_train, image_path_val, mask_path_train, mask_path_val = train_test_split(image_path_subset, mask_path_subset, test_size=val_size)

    train_dataset = tf.data.Dataset.from_tensor_slices((image_path_train, mask_path_train))
    train_dataset = train_dataset.map(load_and_preprocess_data, num_parallel_calls=tf.data.AUTOTUNE)
    #train_dataset = train_dataset.cache()
    #train_dataset = train_dataset.shuffle(1000)
    train_batches = train_dataset.batch(batch_size, drop_remainder=True)#.prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((image_path_val, mask_path_val))
    val_dataset = val_dataset.map(load_and_preprocess_data, num_parallel_calls=tf.data.AUTOTUNE)
    #val_dataset = val_dataset.cache()
    #val_dataset = val_dataset.shuffle(1000)
    val_batches = val_dataset.batch(batch_size, drop_remainder=True)#.prefetch(tf.data.AUTOTUNE)

    # test_dataset = tf.data.Dataset.from_tensor_slices((image_path_test, mask_path_test)) if test_size != 0 else None
    # test_dataset = test_dataset.map(load_and_preprocess_data, num_parallel_calls=tf.data.AUTOTUNE) if test_size != 0 else None
    # test_batches = test_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE) if test_size != 0 else None

    return train_batches, val_batches

train_batches, val_batches = create_datasets(save_path="/home/jupyter/bastianberger/inria1358/raw_data/patches2/", batch_size=2, data_size=0.1)

2023-09-26 09:43:14.771057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-26 09:43:14.786105: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-26 09:43:14.788792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-26 09:43:14.791839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild

In [76]:
train_batches, val_batches

(<BatchDataset element_spec=(TensorSpec(shape=(2, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(2, 512, 512, 1), dtype=tf.float32, name=None))>,
 <BatchDataset element_spec=(TensorSpec(shape=(2, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(2, 512, 512, 1), dtype=tf.float32, name=None))>)

In [4]:
def convolution_block(
    block_input,
    num_filters=256,
    kernel_size=3,
    dilation_rate=1,
    padding="same",
    use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

In [5]:
IMAGE_SIZE, NUM_CLASSES = 512, 2

In [6]:
def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)

In [24]:
class UpdatedMeanIoU(tf.keras.metrics.MeanIoU):
    def __init__(self,
               y_true=None,
               y_pred=None,
               num_classes=None,
               name=None,
               dtype=None):
        super(UpdatedMeanIoU, self).__init__(num_classes = num_classes,name=name, dtype=dtype)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.math.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight)

In [7]:
class UpdatedIoU(tf.keras.metrics.IoU):
    def __init__(self,
               y_true=None,
               y_pred=None,
                 target_class_ids=None,
               num_classes=None,
               name=None,
               dtype=None):
        super(UpdatedIoU, self).__init__(num_classes = num_classes,name=name, dtype=dtype, target_class_ids=target_class_ids)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.math.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight)

In [8]:
def polynomial_decay_lr(iteration, maxiter=1000, power=0.9):
    """Returns the learning rate for the current iteration based on polynomial decay."""
    base_lr = 1.0  # You can set this to your desired initial learning rate
    return base_lr * (1 - (iteration / maxiter)) ** power

# 2. Keras learning rate scheduler
def lr_schedule(epoch, lr):
    """Returns a learning rate based on epoch number."""
    iteration = epoch  # if assuming 1 iteration = 1 epoch, adjust as necessary
    return polynomial_decay_lr(iteration)

# Create a learning rate callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

# 3. Apply this scheduler to your optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=1.0)  # Set to your initial learning rate

# Now, when you fit your model, add the callback:
# model.fit(..., callbacks=[lr_callback])

In [10]:
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
es = tf.keras.callbacks.EarlyStopping(
    patience=2,
    restore_best_weights=True
)
    
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1),
    loss=loss,
    metrics=["accuracy", UpdatedIoU(num_classes=2, target_class_ids=[1])])

In [11]:
history = model.fit(train_batches, validation_data=val_batches, epochs=25, callbacks=[es, lr_callback])

Epoch 1/25


2023-09-26 09:43:55.463978: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200
2023-09-26 09:43:59.136337: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f0c3301d700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-26 09:43:59.136383: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-09-26 09:43:59.141868: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-26 09:43:59.292772: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1219/3543 [=========>....................] - ETA: 7:09 - loss: 0.6829 - accuracy: 0.8361 - updated_io_u_1: 0.0081

KeyboardInterrupt: 

# Running experiments

## Dataset

## Running

## Saving function

In [ ]:
hist_df = pd.DataFrame(history_unet_point1.history)
hist_df.to_csv('./logs/histories/unet_point1.csv')
model_unet_point1.save('.logs/models/unet_point1.keras')

In [9]:
mask_dir = "/home/jupyter/bastianberger/inria1358/raw_data/patches512/train/gt"
mask_path = [os.path.join(mask_dir, filename) for filename in os.listdir(mask_dir)]

In [10]:
from skimage import io
import numpy as np
    
# Check labels for all masks
def check_mask_labels(masks):
    
    # Create an empty set
    unique_labels_len = set()
    
    # Iterate over all mask dataset
    for mask in masks:
        
        # Read mask
        test_mask = tf.io.read_file(mask)
        test_mask = tf.image.decode_png(test_mask, channels=1)
        test_mask = tf.image.resize(test_mask, [512, 512])
        test_mask = tf.math.greater(test_mask, 0)
        #test_mask = tf.cast(test_mask, tf.float32) 
        
        # Find unique labels in the mask
        unique_labels = np.unique(test_mask)
        print(unique_labels)
        
        # Find the total number of unique labels
        len_unique_labels = len(unique_labels)
        
        # Add to the set
        unique_labels_len.add(len_unique_labels)

    # Find the maximum label length
    max_label_len = max(unique_labels_len)
    
    # Convert to list and sort
    unique_labels_len = list(unique_labels_len)
    unique_labels_len.sort()
    
    # Print results
    print(f" Number of labels across all masks: {unique_labels_len} \n Maximum number of masks: {max_label_len}")

    return max_label_len

NUM_CLASSES = check_mask_labels(mask_path)

[False]
[False  True]
[False  True]
[False]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False]
[False  True]
[False  True]
[False  True]
[False  True]
[False]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False]
[False  True]
[False  True]
[False  True]
[False]
[False  True]
[False  True]
[False  True]
[False]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True]
[False  True

KeyboardInterrupt: 